In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')


import copy
import json
from urllib.request import urlretrieve

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from src.blur_face.detection import Detector
from src.blur_face.blurring import  ImageBlur, PixelBlur


annotation_path = '/home/michal/studia/ZPI/data/face_detection.json'
with open(annotation_path, 'r') as file:
    lines = file.readlines()
    annotations = [json.loads(a) for a in lines]

In [ ]:
def download_sample(annotations, sample_size=10):
    sample_indxs = np.random.choice(np.arange(len(annotations)), sample_size, replace=False)
    sample = copy.deepcopy([annotations[i] for i in sample_indxs])
    paths = [urlretrieve(s['content'])[0] for s in sample]
    
    for s, path in zip(sample, paths):
        image = Image.open(path).convert('RGB')
        s['image'] = image
        
    return sample
    
sample = download_sample(annotations)

In [ ]:
from time import time

blur_type = PixelBlur(16)
image_blur = ImageBlur(
    blur_type, 2, 2.5
)

detector = Detector()
for s in sample:
    image = s['image']
    boxes = detector.detect(image)
    image_blurred = image_blur.blur(image, boxes)
    
    plt.figure(figsize=(15, 10))
    plt.imshow(image_blurred)
    plt.show()
    
    plt.savefig(f'figs/{time()}.png')
    
    
